In [2]:
from dap_prinz_green_jobs.getters.ojo_getters import (
    get_mixed_ojo_location_sample,
    get_mixed_ojo_salaries_sample
    
)
from dap_prinz_green_jobs import BUCKET_NAME
from dap_prinz_green_jobs.getters.data_getters import load_s3_data

import pandas as pd
from tqdm import tqdm

## Import and format the data needed

In [3]:
soc_2020_data = pd.read_excel("s3://prinz-green-jobs/inputs/data/occupation_data/ons/extendedsoc2020structureanddescriptionsexcel180523.xlsx",
                         skiprows=1, sheet_name="Extended SOC Framework")

ext_rows = soc_2020_data.dropna(subset=['Sub-Unit Group'])
soc_2020_ext_dict = dict(zip(ext_rows['Sub-Unit Group'], ext_rows['Group Title']))

unit_group_rows = soc_2020_data.dropna(subset=['Unit Group'])
soc_2020_dict = dict(zip(unit_group_rows['Unit Group'], unit_group_rows['Group Title']))

green_occs_outputs = load_s3_data(
        BUCKET_NAME,
        f"outputs/data/ojo_application/extracted_green_measures/20231002/ojo_sample_occupation_green_measures_production_True_base.json",
    )
occs_measures_df = pd.DataFrame.from_dict(green_occs_outputs, orient='index').reset_index().rename(columns={'index':'id'})

for r in ['SOC_2020_EXT', 'SOC_2020', 'SOC_2010']:
    occs_measures_df[r] = occs_measures_df['SOC'].apply(lambda x: x[r] if x else None)


occs_measures_df['SOC_2020_EXT_name'] = occs_measures_df['SOC_2020_EXT'].map(soc_2020_ext_dict)
occs_measures_df['SOC_2020_name'] = occs_measures_df['SOC_2020'].apply(lambda x:soc_2020_dict.get(float(x)) if x else None)

green_inds_outputs = load_s3_data(
        BUCKET_NAME,
        f"outputs/data/ojo_application/extracted_green_measures/20231013/ojo_sample_industry_green_measures_production_True_base_companies_house_False.json",
    )
inds_measures_df = pd.DataFrame.from_dict(green_inds_outputs, orient='index').reset_index().rename(columns={'index':'id'})

green_skills_outputs = load_s3_data(
        BUCKET_NAME,
        f"outputs/data/ojo_application/extracted_green_measures/20230914/ojo_sample_skills_green_measures_production_True_base.json",
    )

skill_measures_df = pd.DataFrame.from_dict(green_skills_outputs, orient='index').reset_index().rename(columns={'index':'id'})

# All skill matches
full_skills_outputs = load_s3_data(
        BUCKET_NAME,
        f"outputs/data/green_skill_lists/20230914/full_esco_skill_mappings.json",
    )



2023-10-19 10:37:12,488 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials
2023-10-19 10:37:13,380 - botocore.credentials - INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [4]:
#we also need wage data, locations info
salary_information = get_mixed_ojo_salaries_sample()
salary_information['id'] = salary_information.id.astype(str)

locations_information = get_mixed_ojo_location_sample()
locations_information['id'] = locations_information.id.astype(str)

In [5]:
#we need soc descriptions as well - they have descriptions for 6 digit sic codes
soc_descriptions = pd.read_excel(
        "s3://prinz-green-jobs/inputs/data/occupation_data/ons/extendedsoc2020structureanddescriptionsexcel121023.xlsx",
        sheet_name="Extended SOC descriptions MG1-9",
    )
soc_descriptions.columns = [i.lower().strip().replace(' ', '_').replace('-', '') for i in soc_descriptions.iloc[0].values]
soc_descriptions.drop(0, inplace=True)
soc_descriptions = (soc_descriptions
                    [['subunit_group', 'group_title', 'descriptions']]
                    #drop na if its na in any of the columns
                    .dropna(subset=['subunit_group', 'group_title', 'descriptions']))
#clean description 
soc_descriptions['clean_description'] = soc_descriptions.descriptions.apply(lambda x: x.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ').replace('  ', ' ').strip().replace('\xa0', ''))
soc_descriptions_dict = soc_descriptions.set_index('subunit_group').clean_description.to_dict()

In [6]:
#function to clean SOC names
def clean_soc_name(soc_name):
    return soc_name.replace('n.e.c.', '').strip()

## Skill information data

In [7]:
ents_explode = skill_measures_df[['id', 'ENTS']].explode("ENTS").reset_index(drop=True)
ents_explode["skill_label"] = ents_explode["ENTS"].apply(lambda x: x[0] if x else [])
ents_explode = ents_explode.explode("skill_label").reset_index(drop=True)

skill_match_thresh = 0.7
extracted_full_skill = []
extracted_full_skill_id = []
for skill_label in tqdm(ents_explode['skill_label']):
    full_skills_output = full_skills_outputs.get(skill_label)
    if full_skills_output and full_skills_output[2]>=skill_match_thresh:
        extracted_full_skill.append(full_skills_output[0])
        extracted_full_skill_id.append(full_skills_output[1])
    else:
        extracted_full_skill.append(None)
        extracted_full_skill_id.append(None)

ents_explode["extracted_full_skill"] = extracted_full_skill
ents_explode["extracted_full_skill_id"] = extracted_full_skill_id

100%|██████████| 789324/789324 [00:00<00:00, 975770.62it/s] 


In [8]:
green_ents_explode = skill_measures_df[['id', 'GREEN_ENTS']].explode("GREEN_ENTS").reset_index(drop=True)
green_ents_explode["skill_label"] = green_ents_explode["GREEN_ENTS"].apply(lambda x: x[0] if isinstance(x, list) else None)
green_ents_explode["extracted_green_skill"] = green_ents_explode["GREEN_ENTS"].apply(
    lambda x: x[1][2][0] if isinstance(x, list) else None)
green_ents_explode["extracted_green_skill_id"] = green_ents_explode["GREEN_ENTS"].apply(
    lambda x: x[1][2][1] if isinstance(x, list) else None)

In [9]:
green_skills_df = pd.concat([ents_explode, green_ents_explode])
green_skills_df = green_skills_df[(
    (green_skills_df["skill_label"]!="") & (pd.notnull(green_skills_df["skill_label"]))
)]

# # Remove the duplicate green skills per job advert
green_skills_df.sort_values(by="extracted_green_skill", inplace=True)
green_skills_df.drop_duplicates(subset=["id", "skill_label"], keep='first', inplace=True)

green_esco_taxonomy = load_s3_data(BUCKET_NAME, "outputs/data/green_skill_lists/green_esco_data_formatted.csv")

full_esco_taxonomy = load_s3_data(
        "open-jobs-lake",
        "escoe_extension/outputs/data/skill_ner_mapping/esco_data_formatted.csv",
    )


green_order_dict = {0: 'preferredLabel', 1: 'altLabels'}
green_esco_taxonomy["type_sort"] = green_esco_taxonomy['type'].map(green_order_dict)
green_esco_taxonomy.sort_values(by="type_sort", inplace=True, ascending=False)
green_esco_taxonomy.drop_duplicates(subset=["id"], keep='first', inplace=True)
green_skill_id_2_name = dict(zip(green_esco_taxonomy['id'], green_esco_taxonomy['description']))

full_order_dict = {0: 'preferredLabel', 1: 'altLabels', 2: 'level_2', 3: 'level_3'}
full_esco_taxonomy["type_sort"] = full_esco_taxonomy['type'].map(green_order_dict)
full_esco_taxonomy.sort_values(by="type_sort", inplace=True, ascending=False)
full_esco_taxonomy.drop_duplicates(subset=["id"], keep='first', inplace=True)
full_skill_id_2_name = dict(zip(full_esco_taxonomy['id'], full_esco_taxonomy['description']))


green_skills_df['full_skill_preferred_name'] = green_skills_df['extracted_full_skill_id'].map(full_skill_id_2_name)
green_skills_df['green_skill_preferred_name'] = green_skills_df['extracted_green_skill_id'].map(green_skill_id_2_name)

## Get the green topics per SOC 2010

In [10]:
from dap_prinz_green_jobs.pipeline.green_measures.occupations.occupations_measures_utils import OccupationMeasures

om = OccupationMeasures()
om.load()

2023-10-19 10:37:41,545 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2023-10-19 10:37:41,980 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device: cpu
2023-10-19 10:37:51,968 - dap_prinz_green_jobs - INFO - Loading SOC job title embeddings
2023-10-19 10:38:00,129 - dap_prinz_green_jobs - INFO - Predict UK SOC for the occupations in the ONET green topics data
2023-10-19 10:38:00,131 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2023-10-19 10:38:00,406 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device: cpu
2023-10-19 10:38:04,822 - dap_prinz_green_jobs - INFO - Loading SOC job title embeddings
2023-10-19 10:38:05,701 - dap_prinz_green_jobs - INFO - Embedding texts in 2.708 batches


0it [00:00, ?it/s]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

1it [00:02,  2.67s/it]

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

3it [00:07,  2.55s/it]

2023-10-19 10:38:13,358 - dap_prinz_green_jobs - INFO - Finding most similar job titles for 1354 job titles



1354it [00:00, 12317.17it/s]

2023-10-19 10:38:13,522 - dap_prinz_green_jobs - INFO - Finding most likely SOC
2023-10-19 10:38:13,524 - dap_prinz_green_jobs - INFO - Found SOCs for 79.83751846381094% of the job titles


## Get SIC 2 digit name

In [11]:
from dap_prinz_green_jobs.getters.industry_getters import load_sic

In [12]:
sic_data = load_sic()

In [13]:
sic_names = dict(zip(sic_data['Division'].tolist(), sic_data['Description'].tolist()))

## Combine it all

In [14]:
print(len(occs_measures_df))
occs_measures_enhanced = occs_measures_df[pd.notnull(occs_measures_df['SOC_2020_EXT_name'])]
occs_measures_enhanced = occs_measures_enhanced.merge(skill_measures_df, on='id')
occs_measures_enhanced = occs_measures_enhanced.merge(inds_measures_df, on='id')
occs_measures_enhanced.loc[occs_measures_enhanced['INDUSTRY GHG PER UNIT EMISSIONS']==':', 'INDUSTRY GHG PER UNIT EMISSIONS'] = None # A curio for 3 rows

occs_measures_enhanced['SIC_2_digit'] = occs_measures_enhanced['SIC'].apply(lambda x: str(x)[0:2] if x else None)
occs_measures_enhanced['SIC_2_digit_name'] = occs_measures_enhanced['SIC_2_digit'].apply(lambda x: sic_names.get(x) if x else None)

occs_measures_enhanced['green_topics_lists'] = occs_measures_enhanced['SOC_2010'].apply(lambda x: om.soc_green_measures_dict.get(x)['ONET_green_topics'])

39866


In [15]:
print(occs_measures_enhanced['SIC'].nunique())
print(occs_measures_enhanced['SIC_2_digit'].nunique())
print(occs_measures_enhanced['SIC_2_digit_name'].nunique())

729
103
88


## Create aggregated dict

In [16]:
occ_unique = occs_measures_enhanced.drop_duplicates(subset=['GREEN CATEGORY', 'GREEN/NOT GREEN', 'GREEN TIMESHARE',
       'GREEN TOPICS', 'SOC_2020_EXT', 'SOC_2020', 'SOC_2010',
       'SOC_2020_EXT_name', 'SOC_2020_name'])
print(len(occ_unique))
print(occs_measures_enhanced['SOC_2020_EXT'].nunique())

882
799


In [17]:
#add salary
occs_measures_enhanced = pd.merge(occs_measures_enhanced, salary_information, on='id', how='left')

#add locations
locations_information = locations_information.drop(columns=['is_uk', 'is_large_geo', 'location', 'coordinates'])
occs_measures_enhanced = pd.merge(occs_measures_enhanced, locations_information, on='id', how='left')

In [18]:
occ_aggregated_data = {}
for SOC_2020_ext in tqdm(occs_measures_enhanced['SOC_2020_EXT'].unique()): 
    if pd.notnull(SOC_2020_ext):
        occ_filtered_data = occs_measures_enhanced[occs_measures_enhanced['SOC_2020_EXT']==SOC_2020_ext]
        
        occ_filtered_skills = green_skills_df[green_skills_df['id'].isin(set(occ_filtered_data['id'].tolist()))]

        top_green_skill_num = occ_filtered_skills[['green_skill_preferred_name', 'extracted_green_skill_id']].value_counts()[0:5].to_dict()
        top_green_skill_prop = occ_filtered_skills[['green_skill_preferred_name', 'extracted_green_skill_id']].value_counts(normalize=True)[0:5].to_dict()

        green_skill_info = []
        for k,v in top_green_skill_num.items():
            green_skill_info.append(
                {"skill_name": k[0], "skill_id": k[1], "num_job_ads": v, "prop_job_ads": top_green_skill_prop[k]})
        
        top_5_not_green_skills_num = occ_filtered_skills[pd.isnull(occ_filtered_skills['extracted_green_skill'])][['full_skill_preferred_name', 'extracted_full_skill_id']].value_counts()[0:5].to_dict()
        top_5_not_green_skills_prop =  occ_filtered_skills[pd.isnull(occ_filtered_skills['extracted_green_skill'])][['full_skill_preferred_name', 'extracted_full_skill_id']].value_counts(normalize=True)[0:5].to_dict()

        not_green_skill_info = []
        for k,v in top_5_not_green_skills_num.items():
            not_green_skill_info.append(
                {"skill_name": k[0], "skill_id": k[1], "num_job_ads": v, "prop_job_ads": top_5_not_green_skills_prop[k]})

        top_5_sics_num = occ_filtered_data[['SIC_2_digit', 'SIC_2_digit_name']].value_counts()[0:5].to_dict()
        top_5_sics_prop = occ_filtered_data[['SIC_2_digit', 'SIC_2_digit_name']].value_counts(normalize=True)[0:5].to_dict()

        top_5_sics = []
        for k,v in top_5_sics_num.items():
            top_5_sics.append({"sic_id": k[0], "sic_name": k[1], "num_job_ads": v, "prop_job_ads": top_5_sics_prop[k]})
        
        top_5_itl2 =  (occ_filtered_data
                           ['itl_2_name']
                           .value_counts()[0:5]
                           .to_dict())

        occ_aggregated_data[SOC_2020_ext] = {
            # General
            'num_job_ads': len(occ_filtered_data),
            'prop_job_ads': len(occ_filtered_data)/len(occs_measures_enhanced),
            'SOC_2020_EXT': SOC_2020_ext,
            'SOC_2020_EXT_name': occ_filtered_data['SOC_2020_EXT_name'].mode()[0] if len(occ_filtered_data['SOC_2020_EXT_name'].mode())!=0 else None,
            'SOC_2020': occ_filtered_data['SOC_2020'].mode()[0] if len(occ_filtered_data['SOC_2020'].mode())!=0 else None,
            'SOC_2010': occ_filtered_data['SOC_2010'].mode()[0] if len(occ_filtered_data['SOC_2010'].mode())!=0 else None,
            # Occupations
            # 'occ_green_category': occ_filtered_data['GREEN CATEGORY'].mode()[0] if len(occ_filtered_data['GREEN CATEGORY'].mode())!=0 else None,
            # 'occ_green_binary': occ_filtered_data['GREEN/NOT GREEN'].mode()[0] if len(occ_filtered_data['GREEN/NOT GREEN'].mode())!=0 else None,
            'occ_timeshare': occ_filtered_data['GREEN TIMESHARE'].mode()[0] if len(occ_filtered_data['GREEN TIMESHARE'].mode())!=0 else None,
            'occ_topics': occ_filtered_data['GREEN TOPICS'].mode()[0] if len(occ_filtered_data['GREEN TOPICS'].mode())!=0 else None,
            'green_topics_lists': occ_filtered_data['green_topics_lists'].iloc[0],
            # Skills
            'average_num_skills': occ_filtered_data['NUM_SPLIT_ENTS'].mean(),
            # 'var_num_skills': occ_filtered_data['NUM_SPLIT_ENTS'].var(skipna=True),
            # 'prop_ads_lt_5_skills': sum(occ_filtered_data['NUM_SPLIT_ENTS']<5)/len(occ_filtered_data),
            'average_prop_green_skills': occ_filtered_data['PROP_GREEN'].mean(),
            # 'var_prop_green_skills': occ_filtered_data['PROP_GREEN'].var(skipna=True),
            'top_5_green_skills': green_skill_info,
            'top_5_not_green_skills': not_green_skill_info,
            # Industry
            'num_unique_SIC2': occ_filtered_data["SIC_2_digit"].nunique(),
            'num_null_sic2': len(occ_filtered_data[pd.isnull(occ_filtered_data["SIC_2_digit"])]),
            'num_top_sic2': occ_filtered_data['SIC_2_digit'].value_counts()[0] if len(occ_filtered_data['SIC_2_digit'].value_counts())>0 else None,
            'num_other_sic2': sum(occ_filtered_data['SIC_2_digit'].value_counts()[1:]) if len(occ_filtered_data['SIC_2_digit'].value_counts())>1 else None,
            'average_ind_perunit_ghg': occ_filtered_data['INDUSTRY GHG PER UNIT EMISSIONS'].mean(),
            'average_ind_prop_hours': occ_filtered_data['INDUSTRY PROP HOURS GREEN TASKS'].mean(),
            'average_ind_prop_workers': occ_filtered_data['INDUSTRY PROP WORKERS GREEN TASKS'].mean(),
            'top_5_sics': top_5_sics,
            #metadata
            ##salary information
            'median_min_annualised_salary': occ_filtered_data.min_annualised_salary.median(),
            'median_max_annualised_salary': occ_filtered_data.max_annualised_salary.median(),
            ##location information 
            'top_5_itl2': {loc: round((num_job_ads/len(occ_filtered_data))*100,2) for loc, num_job_ads in top_5_itl2.items()}
        }

  1%|          | 7/799 [00:00<00:49, 16.12it/s]/Users/india.kerlenesta/opt/anaconda3/envs/sic_test/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/india.kerlenesta/opt/anaconda3/envs/sic_test/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
 16%|█▌        | 125/799 [00:05<00:27, 24.17it/s]/Users/india.kerlenesta/opt/anaconda3/envs/sic_test/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/Users/india.kerlenesta/opt/anaconda3/envs/sic_test/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
 20%|██        | 161/799 [00:07<00:27, 22.86it/s]/Users/india.kerlenesta/opt/anacond

In [19]:
occ_aggregated_data_clean = {}
for soc_code, soc_info in occ_aggregated_data.items():
    loc_info = soc_info['top_5_itl2']
    top_5_locations = list(loc_info.keys())
    top_5_locations_prop = list(loc_info.values())
    soc_desc = soc_descriptions_dict.get(soc_code, None)
    soc_name_cleaned = clean_soc_name(soc_info['SOC_2020_EXT_name'])
    occ_aggregated_data_clean[soc_code] = {
        'num_job_ads': soc_info['num_job_ads'],
        'prop_job_ads': soc_info['prop_job_ads'],
        'SOC_2020_EXT': soc_info['SOC_2020_EXT'],
        'SOC_2020_EXT_name': soc_info['SOC_2020_EXT_name'],
        'clean_soc_name': soc_name_cleaned,
        'soc_description': soc_desc,
        'SOC_2020': soc_info['SOC_2020'],
        'SOC_2010': soc_info['SOC_2010'],
        'occ_timeshare': soc_info['occ_timeshare'],
        'occ_topics': soc_info['occ_topics'],
        'green_topics_lists': soc_info['green_topics_lists'],
        'average_num_skills': soc_info['average_num_skills'],
        'average_prop_green_skills': soc_info['average_prop_green_skills'],
        'top_5_green_skills': soc_info['top_5_green_skills'],
        'num_unique_SIC2': soc_info['num_unique_SIC2'],
        'num_null_sic2': soc_info['num_null_sic2'],
        'num_top_sic2': soc_info['num_top_sic2'],
        'num_other_sic2': soc_info['num_other_sic2'],
        'average_ind_perunit_ghg': soc_info['average_ind_perunit_ghg'],
        'average_ind_prop_hours': soc_info['average_ind_prop_hours'],
        'top_5_sics': soc_info['top_5_sics'],
        'median_min_annualised_salary': soc_info['median_min_annualised_salary'],
        'median_max_annualised_salary': soc_info['median_max_annualised_salary'],
        'top_5_itl2_prop': [soc_info['top_5_itl2']],
    }

## Pull out some examples

In [20]:
occ_aggregated_data = occ_aggregated_data_clean

len(occ_aggregated_data)

799

In [21]:
occ_aggregated_df = pd.DataFrame(occ_aggregated_data).T
occ_aggregated_df = occ_aggregated_df.reset_index()
occ_aggregated_df.rename(columns={'index': 'SOC_2020_EXT'}, inplace=True)
occ_aggregated_df.head(2)

,SOC_2020_EXT,num_job_ads,prop_job_ads,SOC_2020_EXT,SOC_2020_EXT_name,clean_soc_name,soc_description,SOC_2020,SOC_2010,occ_timeshare,...,num_unique_SIC2,num_null_sic2,num_top_sic2,num_other_sic2,average_ind_perunit_ghg,average_ind_prop_hours,top_5_sics,median_min_annualised_salary,median_max_annualised_salary,top_5_itl2_prop
0,2151/02,39,0.001322,2151/02,Ecologists,Ecologists,Ecologists carry out assessments of the health...,2151,2141,57.1,...,7,8,16,15,1.101935,10.829032,"[{'sic_id': '74', 'sic_name': 'Other professio...",30000.0,40000.0,"[{'London': 15.38, 'South Yorkshire': 15.38, '..."
1,4143/99,83,0.002814,4143/99,Customer service managers n.e.c.,Customer service managers,Job holders in this group perform occupations ...,4143,7220,0.0,...,26,14,14,55,0.663134,9.917647,"[{'sic_id': '64', 'sic_name': 'Other financial...",37500.0,42750.0,"[{'London': 14.46, 'Gloucestershire, Wiltshire..."


## Add where this occupation sits
Get IQR boundaries for each green measure and set categorical values for each of them.

In [22]:
big_enough_data = occ_aggregated_df[occ_aggregated_df['num_job_ads']>25]

In [23]:
def categorical_assign(value, all_values, rev=False):

    q1 = all_values.quantile(.33)
    q2 = all_values.quantile(.66)

    if pd.notnull(value):
        if value <= q1:
            if rev:
                return "high"
            else:
                return "low"
        elif value <= q2:
            return "mid"
        else:
            if rev:
                return "low"
            else:
                return "high"
    else:
        return None

In [24]:
big_enough_data['occ_greenness'] = big_enough_data['occ_timeshare'].apply(lambda x: categorical_assign(x, big_enough_data['occ_timeshare']))
big_enough_data['ind_greenness'] = big_enough_data['average_ind_perunit_ghg'].apply(
    lambda x: categorical_assign(x, big_enough_data['average_ind_perunit_ghg'], rev=True))
big_enough_data['skills_greenness'] = big_enough_data['average_prop_green_skills'].apply(lambda x: categorical_assign(x, big_enough_data['average_prop_green_skills']))

/var/folders/cq/9gxjkt2j2g1c0cfhjb2qtz000000gn/T/ipykernel_23618/543948785.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_enough_data['occ_greenness'] = big_enough_data['occ_timeshare'].apply(lambda x: categorical_assign(x, big_enough_data['occ_timeshare']))
/var/folders/cq/9gxjkt2j2g1c0cfhjb2qtz000000gn/T/ipykernel_23618/543948785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_enough_data['ind_greenness'] = big_enough_data['average_ind_perunit_ghg'].apply(
/var/folders/cq/9gxjkt2j2g1c0cfh

In [25]:
def get_one_score(occ,ind,skill):

    score_dict = {"high":2, "mid":1, "low":0}
    
    if occ in score_dict:
        score = score_dict[occ]+ score_dict[ind] + score_dict[skill]
        # return score

        if score <= 1:
            # 0,1
            return "low"
        elif score <=3:
            # 2,3 
            return "low-mid"
        elif score <=5:
            # 4,5
            return "mid-high"
        else:
            # 6
            return "high"
    else:
        return None



In [26]:
big_enough_data["greenness_score"] = big_enough_data.apply(lambda x: get_one_score(x['occ_greenness'], x['ind_greenness'], x['skills_greenness']), axis = 1)

/var/folders/cq/9gxjkt2j2g1c0cfhjb2qtz000000gn/T/ipykernel_23618/1239927552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  big_enough_data["greenness_score"] = big_enough_data.apply(lambda x: get_one_score(x['occ_greenness'], x['ind_greenness'], x['skills_greenness']), axis = 1)


In [27]:
big_enough_data["greenness_score"].value_counts()

low-mid     93
mid-high    73
low         42
high        10
Name: greenness_score, dtype: int64

In [28]:
big_enough_data.columns

Index(['SOC_2020_EXT', 'num_job_ads', 'prop_job_ads', 'SOC_2020_EXT',
       'SOC_2020_EXT_name', 'clean_soc_name', 'soc_description', 'SOC_2020',
       'SOC_2010', 'occ_timeshare', 'occ_topics', 'green_topics_lists',
       'average_num_skills', 'average_prop_green_skills', 'top_5_green_skills',
       'num_unique_SIC2', 'num_null_sic2', 'num_top_sic2', 'num_other_sic2',
       'average_ind_perunit_ghg', 'average_ind_prop_hours', 'top_5_sics',
       'median_min_annualised_salary', 'median_max_annualised_salary',
       'top_5_itl2_prop', 'occ_greenness', 'ind_greenness', 'skills_greenness',
       'greenness_score'],
      dtype='object')

In [29]:
big_enough_data.iloc[0]

SOC_2020_EXT                                                              2151/02
num_job_ads                                                                    39
prop_job_ads                                                             0.001322
SOC_2020_EXT                                                              2151/02
SOC_2020_EXT_name                                                      Ecologists
clean_soc_name                                                         Ecologists
soc_description                 Ecologists carry out assessments of the health...
SOC_2020                                                                     2151
SOC_2010                                                                     2141
occ_timeshare                                                                57.1
occ_topics                                                                     46
green_topics_lists              [Green solutions, Pollution prevention, Eco-to...
average_num_skil

## Save

In [35]:
from datetime import datetime

today = datetime.now().strftime("%Y%m%d")

big_enough_data.to_csv(f"occupation_aggregated_data_{today}.csv")

## Look into

In [37]:
greenness_types = ["low", "low-mid", "mid-high", "high"]
for greenness_type in greenness_types:
    print(f"Greenness type {greenness_type} :")
    print(big_enough_data[big_enough_data["greenness_score"]==greenness_type].sort_values(by = 'num_job_ads', ascending=False)['SOC_2020_EXT_name'].tolist()[0:5])

Greenness type low :
['Software developers', 'Market researchers (excludes interviewers)', 'Student ambassadors', 'Delivery drivers and couriers n.e.c.', 'Transport and distribution clerks and assistants']
Greenness type low-mid :
['Business development managers', 'Bookkeepers, payroll managers and wage clerks n.e.c.', 'Marketing managers', 'Business sales executives', 'Recruitment consultants']
Greenness type mid-high :
['Sustainability officers', 'Business and financial project management professionals n.e.c.', 'Production managers and directors in manufacturing', 'Procurement and purchasing managers and directors', 'Electrical engineers n.e.c.']
Greenness type high :
['Landlords, property and housing managers ', 'Production managers and directors in construction', 'Building and building services engineers', 'Mechanical engineers n.e.c.', 'Mechanical design engineers']


## Quantiles

In [38]:
big_enough_data['occ_timeshare'].quantile(.33)

0.0

In [39]:
big_enough_data['occ_timeshare'].quantile(.66)

16.166

In [45]:
#histogram
#big_enough_data['occ_timeshare'].hist(bins=20)

- Occupation:
- When the values for the % time spent on green tasks per occupation are split equally into thirds, this occupations' value sits in the lowest third.
- When the values for the % time spent on green tasks per occupation are split equally into thirds, this occupations' value sits in the mid third.
- When the values for the % time spent on green tasks per occupation are split equally into thirds, this occupations' value sits in the highest third.

- Industry:
- When the values for the average per unit GHG emissions for industries this occupation sit in are split equally into thirds, this occupations' value sits in the lowest third.
- When the values for the average per unit GHG emissions for industries this occupation sit in are split equally into thirds, this occupations' value sits in the mid third.
- When the values for the average per unit GHG emissions for industries this occupation sit in are split equally into thirds, this occupations' value sits in the highest third.

- Skills:
- When the values for the average proportion of green skills per occupation are split equally into thirds, this occupations' value sits in the lowest third.
- When the values for the average proportion of green skills per occupation are split equally into thirds, this occupations' value sits in the mid third.
- When the values for the average proportion of green skills per occupation are split equally into thirds, this occupations' value sits in the highest third.

In [41]:
big_enough_data['occ_greenness'].value_counts()

low     92
high    74
mid     52
Name: occ_greenness, dtype: int64

In [42]:
big_enough_data[big_enough_data['occ_greenness']=='low']['occ_timeshare'].value_counts()

0.0    92
Name: occ_timeshare, dtype: int64

In [43]:
big_enough_data[big_enough_data['occ_greenness']=='mid']['occ_timeshare'].value_counts()

3.8     5
16.1    4
3.0     3
9.8     3
6.1     3
2.1     2
0.1     2
10.3    2
7.7     2
0.6     2
3.9     1
8.5     1
0.5     1
7.0     1
2.7     1
13.6    1
13.1    1
0.8     1
6.4     1
11.8    1
9.1     1
3.3     1
12.5    1
13.2    1
7.1     1
1.4     1
12.2    1
10.0    1
14.7    1
0.9     1
6.3     1
15.2    1
10.8    1
6.9     1
Name: occ_timeshare, dtype: int64

In [44]:
big_enough_data[big_enough_data['occ_greenness']=='high']['occ_timeshare'].value_counts()

22.4    10
20.8     7
62.5     5
18.8     4
33.3     3
22.5     3
32.5     3
25.3     3
21.1     3
22.6     3
25.1     2
45.3     2
20.0     2
42.1     2
39.0     2
26.2     2
57.1     2
23.1     2
23.4     2
28.9     1
18.5     1
22.1     1
35.6     1
16.4     1
28.6     1
32.1     1
33.8     1
28.8     1
42.9     1
66.7     1
50.0     1
Name: occ_timeshare, dtype: int64